In [1]:
import os
os.chdir(r'/Users/zhanghui/商业分析工具和应用/Python/Python_book/11KNNNB')

In [2]:
import pandas as pd

orgData = pd.read_csv('date_data2.csv')
orgData.head()

,income,attractive,assets,edueduclass,Dated,income_rank,attractive_rank,assets_rank
0,3000,9.0,5.145476,1,0,0,0,0
1,3000,14.5,40.643781,4,1,0,0,1
2,3000,6.0,5.145476,1,0,0,0,0
3,3000,1.0,7.067434,1,0,0,0,0
4,3500,14.5,3.728400,2,0,0,0,0


In [3]:
orgData.describe()

,income,attractive,assets,edueduclass,Dated,income_rank,attractive_rank,assets_rank
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,9010.000000,50.500000,96.006300,3.710000,0.500000,1.550000,1.560000,1.510000
std,5832.675288,28.810948,91.082226,1.225116,0.502519,1.140397,1.103896,1.123621
min,3000.000000,1.000000,3.728400,1.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.000000,28.000000,31.665269,3.000000,0.000000,1.000000,1.000000,0.750000
50%,7500.000000,51.000000,70.746924,4.000000,0.500000,2.000000,2.000000,2.000000
75%,11500.000000,68.875000,131.481061,4.000000,1.000000,3.000000,2.250000,2.250000
max,34000.000000,99.500000,486.311758,6.000000,1.000000,3.000000,3.000000,3.000000


# KNN算法

In [4]:
X = orgData.ix[:,:4]
Y = orgData[['Dated']]

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


In [13]:
#数据标准化，一般距离计算收到数据尺度和量纲影响比较大，所以需要标准化
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)
X_scaled[1:5]

array([[0.        , 0.13705584, 0.07649535, 0.6       ],
       [0.        , 0.05076142, 0.00293644, 0.        ],
       [0.        , 0.        , 0.00691908, 0.        ],
       [0.01612903, 0.13705584, 0.        , 0.2       ]])

In [14]:
#训练集和测试集
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target = train_test_split(
X_scaled,Y,test_size = 0.25,train_size = 0.75,random_state = 1234)

In [17]:
#KNN建模
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 3) #默认欧式距离
model.fit(train_data,train_target.values.flatten())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [20]:
#测试集上评估模型效果
test_est = model.predict(test_data)
import sklearn.metrics as metrics
print(metrics.classification_report(test_target,test_est))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.88      1.00      0.94        15

    accuracy                           0.92        25
   macro avg       0.94      0.90      0.91        25
weighted avg       0.93      0.92      0.92        25



In [21]:
model.score(test_data,test_target)

0.92

# k值的选取
KNN当中除了距离计算方法外，唯一需要确定的参数就是k值。当k较小时，样本寻找与自己最接近的少数几个邻居，这样的模型很不稳定，产生过度拟合。如果k非常大，则预测结果可能会有较大偏差。

In [25]:
#使用k分别取1-9的训练模型，看看他们在预测集上的表现效果
for k in range(1,10):
    k_model = KNeighborsClassifier(n_neighbors = k) 
    k_model.fit(train_data,train_target.values.flatten())
    k_score = k_model.score(test_data,test_target)
    print('when k = %s , the score is %.4f' % (k,k_score))

when k = 1 , the score is 0.9200
when k = 2 , the score is 0.9200
when k = 3 , the score is 0.9200
when k = 4 , the score is 0.9200
when k = 5 , the score is 0.9200
when k = 6 , the score is 0.9200
when k = 7 , the score is 0.9200
when k = 8 , the score is 0.8800
when k = 9 , the score is 0.9200


可以看到，当样本量较小，k的取值与模型得分之间并没有显著规律。
当样本量较小，非常适用于交叉验证评估模型效果。

In [29]:
#使用交叉验证搜索最优参数并且评估
from sklearn.model_selection import ParameterGrid,GridSearchCV

grid = ParameterGrid({'n_neighbors':[range(1,15)]})
estimator = KNeighborsClassifier()
knn_cv = GridSearchCV(estimator,grid,cv=4,scoring = 'roc_auc')
knn_cv.fit(train_data,train_target.values.flatten())

knn_cv.best_params_

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_neighbors': 8}

In [31]:
knn_cv.best_score_

0.949537037037037

In [33]:
#模型在测试集上的评估
metrics.roc_auc_score(test_target,knn_cv.predict(test_data))

0.8666666666666668

# 朴素贝叶斯
适合处理离散型自变量

In [35]:
orgData1 = orgData.ix[:,-3:]
orgData1.head()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,income_rank,attractive_rank,assets_rank
0,0,0,0
1,0,0,1
2,0,0,0
3,0,0,0
4,0,0,0


In [37]:
train_data1,test_data1,train_target1,test_target1 = train_test_split(
orgData1,Y,test_size = 0.3,train_size = 0.7,random_state = 123)

In [38]:
from sklearn.naive_bayes import BernoulliNB

nb = BernoulliNB(alpha =1) #alpha =1 说明用了拉普拉斯平滑，避免出现概率0
nb.fit(train_data1,train_target1.values.flatten())
test_est1 = nb.predict(test_data1)

In [39]:
print(metrics.classification_report(test_target1,test_est1))

              precision    recall  f1-score   support

           0       0.89      0.57      0.70        14
           1       0.71      0.94      0.81        16

    accuracy                           0.77        30
   macro avg       0.80      0.75      0.75        30
weighted avg       0.80      0.77      0.76        30

